In [10]:
import pandas as pd
import numpy as np

df = pd.DataFrame(pd.read_csv('201701_Taiwan.csv'))
df = df[df['device_id']=='74DA3895E010']
df2 = df.sort_values(by=['device_id','Date','Time'])
#split data
df_resample = df2.iloc[:,:8]#Data,Time,device_id,PM2.5,PM10,PM11,Temperature,Humidity
df_location = df2.iloc[:,[2,8,9]].drop_duplicates()#device_id,lat,lon
#datatime index

df_resample=df_resample.set_index(pd.DatetimeIndex(pd.to_datetime(df_resample['Date'] + " " + df_resample['Time'])))
machines=set(df_resample["device_id"])
#resample and split according to device_id
device_dict={}
for machine in machines:
    df_tmp = df_resample[df_resample["device_id"] == machine]
    df_tmp = df_tmp.resample('15T').mean().fillna(method='pad')
    df_tmp.insert(loc=0,column='device_id',value=machine)
    df_tmp.insert(loc=0,column='Time',value=df_tmp.index.time)
    df_tmp.insert(loc=0,column='Date',value=df_tmp.index.date)
    tmp = df_location[df_location['device_id'] == machine]
    df_tmp['lat']=tmp.iloc[0,1]
    df_tmp['lon']=tmp.iloc[0,2]
    device_dict[machine]=df_tmp
df_result=pd.concat(device_dict,ignore_index=True)
df_result = df_result.round({'PM2.5': 0,'PM10':0,'PM1':0,'Temperature':0,'Humidity': 0})
#df_result
df_result.to_csv('/Users/sate/Desktop/DM/hw3/201701.csv')